### All Easy DA - BERT Base Uncased

#### Un-augmented test set
#### Augment only the training set - 10 augmented examples per original

#### Get Original Paper Data

In [1]:
# !pip install sklearn
# !pip install ekphrasis
# !pip install transformers
# !pip install spacy
# !python -m spacy download en_core_web_sm

In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds

import transformers

from transformers import BertTokenizer, TFBertModel
from tensorflow.keras import backend as K
from tensorflow import keras

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

import os

import logging
tf.get_logger().setLevel(logging.ERROR)

In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
encoder = LabelEncoder()
encoder.classes_ = np.load('../Data/classes.npy', allow_pickle=True)

In [5]:
tf.__version__

'2.6.0'

In [6]:
transformers.__version__

'4.16.2'

In [7]:
# test, train, dev examples from base notebook

train_data_df = pd.read_csv('./Saved_Models/EDA_base_uncased_5aug/All_DA_BERT_base_uncased_train_examples.csv')
dev_data_df = pd.read_csv('./Saved_Models/EDA_base_uncased_5aug/All_DA_BERT_base_uncased_dev_examples.csv')
test_data_df = pd.read_csv('./Saved_Models/EDA_base_uncased_5aug/All_DA_BERT_base_uncased_test_examples.csv')

In [8]:
train_data_df.head()

,Unnamed: 0,post_id,text,target1,target2,target3,rationales,final_label,text_combined
0,0,22448349_gab,"['common', 'core', 'weed', 'too', 'much', 'rit...","['Men', 'Women']",['Women'],['None'],[],normal,common core weed too much ritalan chem trails ...
1,1,1178948520201637888_twitter,"['took', 'my', 'nan', 'to', 'the', 'hospital',...",['None'],['None'],['None'],[],normal,took my nan to the hospital for a x ray i turn...
2,2,1482573_gab,"['<user>', 'well', 'not', 'really', 'islam', '...",['Islam'],['Other'],['Islam'],"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,...",offensive,<user> well not really islam does not care for...
3,3,1097184028149587969_twitter,"['<user>', 'france', 'in', '<number>', 'after'...","['Islam', 'Other']",['Islam'],['Islam'],[],normal,<user> france in <number> after muslims take o...
4,4,1089569255111176192_twitter,"['i', 'will', 'not', 'tolerate', 'non', 'arab'...","['Arab', 'Men', 'Women']",['Arab'],"['Arab', 'Islam']","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,...",hatespeech,i will not tolerate non arab women slandering ...


In [9]:
X_train_id = train_data_df['post_id']
X_test_id = test_data_df['post_id']
X_dev_id = dev_data_df['post_id']

In [10]:
y_train = train_data_df['final_label']
y_test = test_data_df['final_label']
y_dev = dev_data_df['final_label']

In [11]:
x_train_df = pd.DataFrame({'post_id' : X_train_id.to_list()})
x_dev_df = pd.DataFrame({'post_id' : X_dev_id.to_list()})
x_test_df = pd.DataFrame({'post_id' : X_test_id.to_list()})

# X_train_df = pd.merge(x_train_df, raw_data_final, how='inner', on='post_id')
# X_dev_df = pd.merge(x_dev_df, raw_data_final, how='inner', on='post_id')
# X_test_df = pd.merge(x_test_df, raw_data_final, how='inner', on='post_id')

X_train_text = train_data_df['text_combined'].to_list()
X_dev_text= dev_data_df['text_combined'].to_list()
X_test_text = test_data_df['text_combined'].to_list()

print(len(X_train_text))
print(len(X_dev_text))
print(len(X_test_text))

15383
1923
1923


### Get Augmented Data

In [12]:
# load augmented datasets generated by EDA
# sr = synonym replacement
# ri = random synonym insertion
# rs = random swap
# rd = random deletion
# dataframe name format: method_number 

sr_1_df = pd.read_csv('../test_data_set/EDA_10_0_7_sr_rest_0_1.csv')
ri_1_df = pd.read_csv('../test_data_set/EDA_10_0_7_ri_rest_0_1.csv')
rs_1_df = pd.read_csv('../test_data_set/EDA_10_0_7_rs_rest_0_1.csv')
rd_1_df = pd.read_csv('../test_data_set/EDA_10_0_7_rd_rest_0_1.csv')
all_1_df = pd.read_csv('../test_data_set/EDA_10_all_0_1s.csv')
all_5_df = pd.read_csv('../test_data_set/EDA_10_all_0_5s.csv')

# remove undecided labeled examples
sr_1_df_filtered = sr_1_df[sr_1_df['final_label'] != 'undecided']
ri_1_df_filtered = ri_1_df[ri_1_df['final_label'] != 'undecided']
rs_1_df_filtered = rs_1_df[rs_1_df['final_label'] != 'undecided']
rd_1_df_filtered = rd_1_df[rd_1_df['final_label'] != 'undecided']
all_1_df_filtered = all_1_df[all_1_df['final_label'] != 'undecided']
all_5_df_filtered = all_5_df[all_5_df['final_label'] != 'undecided']

len(sr_1_df_filtered)

211519

In [13]:
# separate train, dev, test for each set
sr_1_df_train = sr_1_df_filtered[sr_1_df_filtered['post_id'].isin(X_train_id)]
ri_1_df_train = ri_1_df_filtered[ri_1_df_filtered['post_id'].isin(X_train_id)]
rs_1_df_train = rs_1_df_filtered[rs_1_df_filtered['post_id'].isin(X_train_id)]
rd_1_df_train = rd_1_df_filtered[rd_1_df_filtered['post_id'].isin(X_train_id)]
all_1_df_train = all_1_df_filtered[all_1_df_filtered['post_id'].isin(X_train_id)]
all_5_df_train = all_5_df_filtered[all_5_df_filtered['post_id'].isin(X_train_id)]

# select text sets

aug_sr_text = sr_1_df_train['text_str'].to_list()
aug_ri_text = ri_1_df_train['text_str'].to_list()
aug_rs_text = rs_1_df_train['text_str'].to_list()
aug_rd_text = rd_1_df_train['text_str'].to_list()
aug_all_1_text = all_1_df_train['text_str'].to_list()
aug_all_5_text = all_5_df_train['text_str'].to_list()

# select label sets

aug_sr_labels = sr_1_df_train['final_label']
aug_ri_labels = ri_1_df_train['final_label']
aug_rs_labels = rs_1_df_train['final_label']
aug_rd_labels = rd_1_df_train['final_label']
aug_all_1_labels = all_1_df_train['final_label']
aug_all_5_labels = all_5_df_train['final_label']

len(aug_sr_text)

169213

#### Convert labels to one-hot encoding

In [14]:
# convert class label to 1 hot encoding

def convert_to_oh(S):
    '''takes a pandas series of text labels and returns one hot encoding equivalent
    0 = normal, 1 = offensive, 2 = hatespeech
    ''' 
    S_numerical = S.apply(lambda x: 0 if x=='normal' else (1 if x=='offensive' else 2))
    S_oh = keras.utils.to_categorical(S_numerical, num_classes = 3, dtype = 'float32')
    return S_oh
    
# original dataset - train, dev, and train
y_train_orig = convert_to_oh(pd.Series(y_train))
y_dev_orig = convert_to_oh(pd.Series(y_dev))
y_test_orig = convert_to_oh(pd.Series(y_test))

# augmented with sr = 0.1
y_train_aug_sr = convert_to_oh(aug_sr_labels)

# augmented with ri = 0.1
y_train_aug_ri = convert_to_oh(aug_ri_labels)

# augmented with rs = 0.1
y_train_aug_rs = convert_to_oh(aug_rs_labels)

# augmented with rd = 0.1
y_train_aug_rd = convert_to_oh(aug_rd_labels)

# augmented with all = 0.1
y_train_all_1 = convert_to_oh(aug_all_1_labels)

# augmented with all = 0.5
y_train_all_5 = convert_to_oh(aug_all_5_labels)


In [15]:
len(aug_ri_text)

169213

In [16]:
len(y_train_aug_ri)

169213

In [17]:
y_train_aug_ri

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]], dtype=float32)

### BERT Model

In [18]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#bert_model = TFBertModel.from_pretrained('bert-base-uncased')

In [19]:
aug_sr_text

['i dont think im getting my baby them white he has two white j and nikes not even touched',
 'dont think getting baby them white he has two white nikes not even touched',
 'i dont think im getting my baby j white he has two white them and nikes not even touched',
 'i dont consider im commence my mollycoddle them ashen he has ii ashen joule and nike not tied bear upon',
 'i dont think im getting my baby them white he has two white j and nikes even not touched',
 'i dont think im getting my baby them white he has two white j and nikes affect not even touched',
 'i dont imagine im incur my spoil them ovalbumin he has deuce ovalbumin joule and nike not evening impact',
 'i dont think getting my baby them white he has two white j nikes not even touched',
 'i dont think im getting my baby them white he has two white j and beget nikes not even touched',
 'i dont think im getting my baby them white he has nikes white j and two not even touched',
 'i dont think im getting my baby them white he

In [20]:
max_length = 128

def bert_tokenize(train_set, dev_set, test_set, max_length):
    
    train = tokenizer(train_set, max_length=max_length, truncation=True, padding='max_length', return_tensors='tf')
    dev = tokenizer(dev_set, max_length=max_length, truncation=True, padding='max_length', return_tensors='tf')
    test = tokenizer(test_set, max_length=max_length, truncation=True, padding='max_length', return_tensors='tf')
    
    return train, dev, test

X_train_orig, X_dev_orig, X_test_orig = bert_tokenize(X_train_text, X_dev_text, X_test_text, max_length)

X_train_aug_sr, X_dev_aug_sr, X_test_aug_sr = bert_tokenize(aug_sr_text, X_dev_text, X_test_text, max_length)

X_train_aug_ri, X_dev_aug_ri, X_test_aug_ri = bert_tokenize(aug_ri_text, X_dev_text, X_test_text, max_length)

X_train_aug_rs, X_dev_aug_rs, X_test_aug_rs = bert_tokenize(aug_rs_text, X_dev_text, X_test_text, max_length)

X_train_aug_rd, X_dev_aug_rd, X_test_aug_rd = bert_tokenize(aug_rd_text, X_dev_text, X_test_text, max_length)

X_train_all_1, X_dev_all_1, X_test_all_1 = bert_tokenize(aug_all_1_text, X_dev_text, X_test_text, max_length)

X_train_all_5, X_dev_all_5, X_test_all_5 = bert_tokenize(aug_all_5_text, X_dev_text, X_test_text, max_length)


In [21]:
#tokenizer.save_pretrained("./Tokenizer_ALL_EDA_BERT_base_uncased")

In [22]:
X_train_orig.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [23]:
X_train_aug_sr.input_ids

<tf.Tensor: shape=(169213, 128), dtype=int32, numpy=
array([[  101,  1045,  2123, ...,     0,     0,     0],
       [  101,  2123,  2102, ...,     0,     0,     0],
       [  101,  1045,  2123, ...,     0,     0,     0],
       ...,
       [  101,  1996,  3644, ...,     0,     0,     0],
       [  101,  1996, 18414, ...,     0,     0,     0],
       [  101,  1996,  3644, ...,     0,     0,     0]])>

In [24]:
X_train_aug_ri.token_type_ids

<tf.Tensor: shape=(169213, 128), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])>

In [25]:
X_train_all_1.attention_mask

<tf.Tensor: shape=(169213, 128), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])>

In [26]:
X_train_all_5.input_ids

<tf.Tensor: shape=(169213, 128), dtype=int32, numpy=
array([[ 101, 1045, 2123, ...,    0,    0,    0],
       [ 101, 1045, 5028, ...,    0,    0,    0],
       [ 101, 2068, 1045, ...,    0,    0,    0],
       ...,
       [ 101, 1996, 7486, ...,    0,    0,    0],
       [ 101, 3644, 7069, ...,    0,    0,    0],
       [ 101, 1996, 3644, ...,    0,    0,    0]])>

In [27]:
from keras import backend as K

def balanced_recall(y_true, y_pred):
    """This function calculates the balanced recall metric
    recall = TP / (TP + FN)
    """
    recall_by_class = 0
    # iterate over each predicted class to get class-specific metric
    for i in range(y_pred.shape[1]):
        y_pred_class = y_pred[:, i]
        y_true_class = y_true[:, i]
        true_positives = K.sum(K.round(K.clip(y_true_class * y_pred_class, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true_class, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        recall_by_class = recall_by_class + recall
    return recall_by_class / y_pred.shape[1]

def balanced_precision(y_true, y_pred):
    """This function calculates the balanced precision metric
    precision = TP / (TP + FP)
    """
    precision_by_class = 0
    # iterate over each predicted class to get class-specific metric
    for i in range(y_pred.shape[1]):
        y_pred_class = y_pred[:, i]
        y_true_class = y_true[:, i]
        true_positives = K.sum(K.round(K.clip(y_true_class * y_pred_class, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred_class, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        precision_by_class = precision_by_class + precision
    # return average balanced metric for each class
    return precision_by_class / y_pred.shape[1]

def balanced_f1_score(y_true, y_pred):
    """This function calculates the F1 score metric"""
    precision = balanced_precision(y_true, y_pred)
    recall = balanced_recall(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

In [28]:
def create_classification_model(bert_model, hidden_size = 5, 
                                train_layers = -1, 
                                optimizer=tf.keras.optimizers.Adam()):
    """
    Build a simple classification model with BERT. Let's keep it simple and don't add dropout, layer norms, etc.
    """

    input_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='input_ids_layer')
    token_type_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='token_type_ids_layer')
    attention_mask = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int32, name='attention_mask_layer')

    bert_inputs = {'input_ids': input_ids,
                  'token_type_ids': token_type_ids,
                  'attention_mask': attention_mask}


    #restrict training to the train_layers outer transformer layers
    if not train_layers == -1:

            retrain_layers = []

            for retrain_layer_number in range(train_layers):

                layer_code = '_' + str(11 - retrain_layer_number)
                retrain_layers.append(layer_code)

            for w in bert_model.weights:
                if not any([x in w.name for x in retrain_layers]):
                    w._trainable = False


    bert_out = bert_model(bert_inputs)
    
    net = bert_out[0]
    
    classification_token = tf.keras.layers.Lambda(lambda x: x[:,0,:], name='get_first_vector')(net)
    
    dropout1 = tf.keras.layers.Dropout(0.4, name="dropout1")(classification_token)
    
    hidden = tf.keras.layers.Dense(hidden_size, name='hidden_layer')(dropout1)
    
    dropout2 = tf.keras.layers.Dropout(0.4, name="dropout2")(hidden)

    classification = tf.keras.layers.Dense(3, activation='sigmoid',name='classification_layer')(dropout2)

    classification_model = tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask], 
                                          outputs=[classification])
    
    METRICS = [tf.keras.metrics.CategoricalAccuracy(name="accuracy"), 
               balanced_recall, 
               balanced_precision, 
               balanced_f1_score,
               tf.keras.metrics.AUC(curve='ROC', name="auc_roc")]
    
    
    classification_model.compile(optimizer=optimizer,
                            loss=tf.keras.losses.CategoricalCrossentropy(),
                            metrics= METRICS)


    return classification_model




#     classification_model.compile(optimizer=optimizer,
#                             loss=tf.keras.losses.CategoricalCrossentropy(),
#                             metrics=tf.keras.metrics.CategoricalAccuracy('accuracy'))

In [29]:
def fine_tune_BERT(x_train, x_dev, x_test, y_train, y_dev, y_test, name, learning_rate = 5e-05, 
                   epsilon=1e-08, train_layers = -1, epochs = 10, batch_size = 16):
    ''' Fine tunes BERT base uncased with given data, allows your to set some hyperparameters
        returns test set accuracy, f1 score, and AUC_ROC score
    '''
    try:
        del classification_model
    except:
        pass

    try:
        del bert_model
    except:
        pass
    
    tf.keras.backend.clear_session()
    bert_model = TFBertModel.from_pretrained('bert-base-uncased')

    # early stopping callback
    
    earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy', 
                                                      patience = 4,
                                                      restore_best_weights = True)
    
    # Create a callback that saves the model's weights
    
    path_name = './Saved_Models/EDA_b_10aug/' + name + '/' + name

    cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=path_name, 
                                                     save_weights_only=True,
                                                     verbose=1,
                                                     monitor='val_accuracy',
                                                     save_best_only=True)
    
    # create classification model
    classification_model = create_classification_model(bert_model, 
                                                       optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=epsilon),
                                                       train_layers=train_layers)    
    
    model_fit = classification_model.fit([x_train.input_ids, x_train.token_type_ids, x_train.attention_mask],
                         y_train,
                         validation_data=([x_dev.input_ids, x_dev.token_type_ids, x_dev.attention_mask],
                         y_dev),
                        epochs=epochs,
                        batch_size=batch_size,
                        callbacks = [earlystop_callback, cp_callback])
    
    y_preds_array = classification_model.predict([x_test.input_ids, x_test.token_type_ids, x_test.attention_mask])

    # convert to predicted one-hot encoding

    from keras.utils.np_utils import to_categorical
    y_preds = to_categorical(np.argmax(y_preds_array, 1), dtype = "int64")

    # convert back to labels

    y_test_cat = np.argmax(y_test, axis=1)
    y_preds_cat = np.argmax(y_preds, axis=1)
    
    # calculate metrics
    Accuracy = accuracy_score(y_test_cat, y_preds_cat)

    Macro_F1 = f1_score(y_test_cat, y_preds_cat, average='macro')

    ROC_AUC = roc_auc_score(y_test, y_preds, multi_class='ovo',average='macro')
    
    metrics_history = model_fit.history
    
    return Accuracy, Macro_F1, ROC_AUC, metrics_history

In [49]:
%%time
# original data set
Accuracy_orig, Macro_F1_orig, ROC_AUC_orig, metrics_orig = fine_tune_BERT(X_train_orig, X_dev_orig, X_test_orig, 
                                                            y_train_orig, y_dev_orig, y_test_orig, 'orig_data_10aug',
                                                            learning_rate = 2e-05, epsilon=1e-08, 
                                                            train_layers = 1, epochs = 30, batch_size = 64)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/30
241/241 [==============================] - 59s 201ms/step - loss: 1.3042 - accuracy: 0.4132 - balanced_recall: 0.5771 - balanced_precision: 0.3823 - balanced_f1_score: 0.4596 - auc_roc: 0.5877 - val_loss: 0.9417 - val_accuracy: 0.5367 - val_balanced_recall: 0.7150 - val_balanced_precision: 0.4686 - val_balanced_f1_score: 0.5649 - val_auc_roc: 0.7279

Epoch 00001: val_accuracy improved from -inf to 0.53666, saving model to ./Saved_Models/EDA_b_10aug/orig_data_10aug\orig_data_10aug
Epoch 2/30
241/241 [==============================] - 44s 182ms/step - loss: 1.1072 - accuracy: 0.4789 - balanced_recall: 0.6480 - balanced_precision: 0.4274 - balanced_f1_score: 0.5148 - auc_roc: 0.6620 - val_loss: 0.8825 - val_accuracy: 0.5793 - val_balanced_recall: 0.7308 - val_balanced_precision: 0.5106 - val_balanced_f1_score: 0.6002 - val_auc_roc: 0.7658

Epoch 00002: val_accuracy improved from 0.53666 to 0.57930, saving model to ./Saved_Models/EDA_b_10aug/orig_data_10aug\orig_data_10aug
Epoch

241/241 [==============================] - 44s 184ms/step - loss: 0.7998 - accuracy: 0.6445 - balanced_recall: 0.7954 - balanced_precision: 0.5444 - balanced_f1_score: 0.6460 - auc_roc: 0.8087 - val_loss: 0.7337 - val_accuracy: 0.6760 - val_balanced_recall: 0.8040 - val_balanced_precision: 0.5736 - val_balanced_f1_score: 0.6687 - val_auc_roc: 0.8423

Epoch 00018: val_accuracy did not improve from 0.68071
Epoch 19/30
241/241 [==============================] - 43s 179ms/step - loss: 0.7817 - accuracy: 0.6516 - balanced_recall: 0.8002 - balanced_precision: 0.5457 - balanced_f1_score: 0.6485 - auc_roc: 0.8143 - val_loss: 0.7500 - val_accuracy: 0.6760 - val_balanced_recall: 0.7994 - val_balanced_precision: 0.5711 - val_balanced_f1_score: 0.6653 - val_auc_roc: 0.8399

Epoch 00019: val_accuracy did not improve from 0.68071
Epoch 20/30
241/241 [==============================] - 44s 181ms/step - loss: 0.7874 - accuracy: 0.6564 - balanced_recall: 0.8008 - balanced_precision: 0.5444 - balanced_f1

In [31]:
%%time
# augmented with sr = 0.1
Accuracy_aug_sr, Macro_F1_aug_sr, ROC_AUC_aug_sr, metrics_sr = fine_tune_BERT(X_train_aug_sr, X_dev_aug_sr, X_test_aug_sr, 
                                                            y_train_aug_sr, y_dev_orig, y_test_orig, 'EDA_sr_10aug', 
                                                            learning_rate = 2e-05, epsilon=1e-08, 
                                                            train_layers = 1, epochs = 30, batch_size = 64)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/30
2644/2644 [==============================] - 442s 163ms/step - loss: 1.0198 - accuracy: 0.5068 - balanced_recall: 0.6147 - balanced_precision: 0.4307 - balanced_f1_score: 0.5057 - auc_roc: 0.6584 - val_loss: 0.7889 - val_accuracy: 0.6599 - val_balanced_recall: 0.7246 - val_balanced_precision: 0.5388 - val_balanced_f1_score: 0.6177 - val_auc_roc: 0.7911

Epoch 00001: val_accuracy improved from -inf to 0.65991, saving model to ./Saved_Models/EDA_b_10aug/EDA_sr_10aug\EDA_sr_10aug
Epoch 2/30
2644/2644 [==============================] - 425s 161ms/step - loss: 0.8770 - accuracy: 0.5949 - balanced_recall: 0.7247 - balanced_precision: 0.4860 - balanced_f1_score: 0.5812 - auc_roc: 0.7379 - val_loss: 0.7467 - val_accuracy: 0.6823 - val_balanced_recall: 0.7703 - val_balanced_precision: 0.5492 - val_balanced_f1_score: 0.6408 - val_auc_roc: 0.8154

Epoch 00002: val_accuracy improved from 0.65991 to 0.68227, saving model to ./Saved_Models/EDA_b_10aug/EDA_sr_10aug\EDA_sr_10aug
Epoch 3/30


In [32]:
%%time
# augmented with ri = 0.1
Accuracy_aug_ri, Macro_F1_aug_ri, ROC_AUC_aug_ri, metrics_ri = fine_tune_BERT(X_train_aug_ri, X_dev_aug_ri, X_test_aug_ri, 
                                                            y_train_aug_ri, y_dev_orig, y_test_orig, 'EDA_ri_10aug', 
                                                            learning_rate = 2e-05, epsilon=1e-08, 
                                                            train_layers = 1, epochs = 30, batch_size = 64)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/30
2644/2644 [==============================] - 435s 161ms/step - loss: 1.0089 - accuracy: 0.5317 - balanced_recall: 0.6453 - balanced_precision: 0.4556 - balanced_f1_score: 0.5333 - auc_roc: 0.6875 - val_loss: 0.7744 - val_accuracy: 0.6604 - val_balanced_recall: 0.7421 - val_balanced_precision: 0.5357 - val_balanced_f1_score: 0.6218 - val_auc_roc: 0.7954

Epoch 00001: val_accuracy improved from -inf to 0.66043, saving model to ./Saved_Models/EDA_b_10aug/EDA_ri_10aug\EDA_ri_10aug
Epoch 2/30
2644/2644 [==============================] - 426s 161ms/step - loss: 0.8519 - accuracy: 0.6192 - balanced_recall: 0.6964 - balanced_precision: 0.5045 - balanced_f1_score: 0.5843 - auc_roc: 0.7495 - val_loss: 0.7489 - val_accuracy: 0.6812 - val_balanced_recall: 0.7802 - val_balanced_precision: 0.5427 - val_balanced_f1_score: 0.6397 - val_auc_roc: 0.7953

Epoch 00002: val_accuracy improved from 0.66043 to 0.68123, saving model to ./Saved_Models/EDA_b_10aug/EDA_ri_10aug\EDA_ri_10aug
Epoch 3/30


In [33]:
%%time
# augmented with rs = 0.1
Accuracy_aug_rs, Macro_F1_aug_rs, ROC_AUC_aug_rs, metrics_rs = fine_tune_BERT(X_train_aug_rs, X_dev_aug_rs, X_test_aug_rs, 
                                                            y_train_aug_rs, y_dev_orig, y_test_orig, 'EDA_rs_10aug',
                                                            learning_rate = 2e-05, epsilon=1e-08, 
                                                            train_layers = 1, epochs = 30, batch_size = 64)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/30
2644/2644 [==============================] - 433s 161ms/step - loss: 0.9808 - accuracy: 0.5306 - balanced_recall: 0.7234 - balanced_precision: 0.4385 - balanced_f1_score: 0.5454 - auc_roc: 0.6939 - val_loss: 0.7696 - val_accuracy: 0.6682 - val_balanced_recall: 0.8490 - val_balanced_precision: 0.5248 - val_balanced_f1_score: 0.6472 - val_auc_roc: 0.8149

Epoch 00001: val_accuracy improved from -inf to 0.66823, saving model to ./Saved_Models/EDA_b_10aug/EDA_rs_10aug\EDA_rs_10aug
Epoch 2/30
2644/2644 [==============================] - 425s 161ms/step - loss: 0.8438 - accuracy: 0.6120 - balanced_recall: 0.7672 - balanced_precision: 0.4826 - balanced_f1_score: 0.5918 - auc_roc: 0.7518 - val_loss: 0.7441 - val_accuracy: 0.6713 - val_balanced_recall: 0.7967 - val_balanced_precision: 0.5598 - val_balanced_f1_score: 0.6567 - val_auc_roc: 0.8222

Epoch 00002: val_accuracy improved from 0.66823 to 0.67135, saving model to ./Saved_Models/EDA_b_10aug/EDA_rs_10aug\EDA_rs_10aug
Epoch 3/30


In [34]:
%%time
# augmented with rd = 0.1
Accuracy_aug_rd, Macro_F1_aug_rd, ROC_AUC_aug_rd, metrics_rd = fine_tune_BERT(X_train_aug_rd, X_dev_aug_rd, X_test_aug_rd, 
                                                            y_train_aug_rd, y_dev_orig, y_test_orig, 'EDA_rd_10aug',
                                                            learning_rate = 2e-05, epsilon=1e-08, 
                                                            train_layers = 1, epochs = 30, batch_size = 64)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/30
2644/2644 [==============================] - 433s 161ms/step - loss: 1.0639 - accuracy: 0.4975 - balanced_recall: 0.6618 - balanced_precision: 0.4138 - balanced_f1_score: 0.5084 - auc_roc: 0.6464 - val_loss: 0.8127 - val_accuracy: 0.6485 - val_balanced_recall: 0.8088 - val_balanced_precision: 0.5143 - val_balanced_f1_score: 0.6258 - val_auc_roc: 0.7776

Epoch 00001: val_accuracy improved from -inf to 0.64847, saving model to ./Saved_Models/EDA_b_10aug/EDA_rd_10aug\EDA_rd_10aug
Epoch 2/30
2644/2644 [==============================] - 425s 161ms/step - loss: 0.9148 - accuracy: 0.5723 - balanced_recall: 0.7318 - balanced_precision: 0.4423 - balanced_f1_score: 0.5507 - auc_roc: 0.7018 - val_loss: 0.7648 - val_accuracy: 0.6755 - val_balanced_recall: 0.8480 - val_balanced_precision: 0.5089 - val_balanced_f1_score: 0.6328 - val_auc_roc: 0.7928

Epoch 00002: val_accuracy improved from 0.64847 to 0.67551, saving model to ./Saved_Models/EDA_b_10aug/EDA_rd_10aug\EDA_rd_10aug
Epoch 3/30


In [35]:
%%time
# augmented with all = 0.1
Accuracy_aug_all_1, Macro_F1_aug_all_1, ROC_AUC_aug_all_1, metrics_all_1 = fine_tune_BERT(X_train_all_1, X_dev_all_1, X_test_all_1, 
                                                            y_train_all_1, y_dev_orig, y_test_orig, 'EDA_all_1_10aug',
                                                            learning_rate = 2e-05, epsilon=1e-08, 
                                                            train_layers = 1, epochs = 30, batch_size = 64)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/30
2644/2644 [==============================] - 433s 161ms/step - loss: 0.9484 - accuracy: 0.5494 - balanced_recall: 0.6916 - balanced_precision: 0.3838 - balanced_f1_score: 0.4927 - auc_roc: 0.6134 - val_loss: 0.7802 - val_accuracy: 0.6708 - val_balanced_recall: 0.8796 - val_balanced_precision: 0.4290 - val_balanced_f1_score: 0.5751 - val_auc_roc: 0.7782

Epoch 00001: val_accuracy improved from -inf to 0.67083, saving model to ./Saved_Models/EDA_b_10aug/EDA_all_1_10aug\EDA_all_1_10aug
Epoch 2/30
2644/2644 [==============================] - 425s 161ms/step - loss: 0.8239 - accuracy: 0.6365 - balanced_recall: 0.7258 - balanced_precision: 0.3995 - balanced_f1_score: 0.5145 - auc_roc: 0.6455 - val_loss: 0.7568 - val_accuracy: 0.6838 - val_balanced_recall: 0.8662 - val_balanced_precision: 0.4594 - val_balanced_f1_score: 0.5990 - val_auc_roc: 0.7980

Epoch 00002: val_accuracy improved from 0.67083 to 0.68383, saving model to ./Saved_Models/EDA_b_10aug/EDA_all_1_10aug\EDA_all_1_10aug

In [36]:
%%time
# augmented with all = 0.5
Accuracy_aug_all_5, Macro_F1_aug_all_5, ROC_AUC_aug_all_5, metrics_all_5 = fine_tune_BERT(X_train_all_5, X_dev_all_5, X_test_all_5, 
                                                            y_train_all_5, y_dev_orig, y_test_orig, 'EDA_all_5_10aug',
                                                            learning_rate = 2e-05, epsilon=1e-08, 
                                                            train_layers = 1, epochs = 30, batch_size = 64)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/30
2644/2644 [==============================] - 433s 161ms/step - loss: 1.0467 - accuracy: 0.4857 - balanced_recall: 0.5229 - balanced_precision: 0.4363 - balanced_f1_score: 0.4742 - auc_roc: 0.6370 - val_loss: 0.7944 - val_accuracy: 0.6438 - val_balanced_recall: 0.7012 - val_balanced_precision: 0.5735 - val_balanced_f1_score: 0.6301 - val_auc_roc: 0.8065

Epoch 00001: val_accuracy improved from -inf to 0.64379, saving model to ./Saved_Models/EDA_b_10aug/EDA_all_5_10aug\EDA_all_5_10aug
Epoch 2/30
2644/2644 [==============================] - 423s 160ms/step - loss: 0.9230 - accuracy: 0.5689 - balanced_recall: 0.6109 - balanced_precision: 0.4691 - balanced_f1_score: 0.5292 - auc_roc: 0.6875 - val_loss: 0.7635 - val_accuracy: 0.6667 - val_balanced_recall: 0.7648 - val_balanced_precision: 0.5480 - val_balanced_f1_score: 0.6373 - val_auc_roc: 0.8068

Epoch 00002: val_accuracy improved from 0.64379 to 0.66667, saving model to ./Saved_Models/EDA_b_10aug/EDA_all_5_10aug\EDA_all_5_10aug

In [58]:
Accuracy_orig, Macro_F1_orig, ROC_AUC_orig, metrics_orig

(0.703068122724909,
 0.6830480350859433,
 0.7660838421593968,
 {'loss': [1.3042175769805908,
   1.1071772575378418,
   1.0218576192855835,
   0.9753274917602539,
   0.9385999441146851,
   0.916458785533905,
   0.9061144590377808,
   0.8801248073577881,
   0.8665584921836853,
   0.8640263080596924,
   0.8530023694038391,
   0.8423669338226318,
   0.8295612335205078,
   0.8290104269981384,
   0.8134371042251587,
   0.8112388253211975,
   0.8006798028945923,
   0.7997586727142334,
   0.7816831469535828,
   0.7874255776405334,
   0.7729077935218811,
   0.7625702619552612,
   0.7577171325683594,
   0.7567818760871887,
   0.7422648668289185],
  'accuracy': [0.41318339109420776,
   0.4789052903652191,
   0.5170642733573914,
   0.5486575961112976,
   0.5652993321418762,
   0.5792108178138733,
   0.5849314332008362,
   0.6017031669616699,
   0.6070337295532227,
   0.6091789603233337,
   0.6165897250175476,
   0.6254956722259521,
   0.6275758743286133,
   0.6322563886642456,
   0.634531617164611

In [59]:
# Accuracy_aug_sr, Macro_F1_aug_sr, ROC_AUC_aug_sr, metrics_sr

In [60]:
# Accuracy_aug_ri, Macro_F1_aug_ri, ROC_AUC_aug_ri, metrics_ri

In [61]:
# Accuracy_aug_rs, Macro_F1_aug_rs, ROC_AUC_aug_rs, metrics_rs

In [62]:
# Accuracy_aug_rd, Macro_F1_aug_rd, ROC_AUC_aug_rd, metrics_rd

In [63]:
# Accuracy_aug_all_1, Macro_F1_aug_all_1, ROC_AUC_aug_all_1, metrics_all_1

In [64]:
# Accuracy_aug_all_5, Macro_F1_aug_all_5, ROC_AUC_aug_all_5, metrics_all_5

In [65]:
trial_name_list = ['Original Data', 'Augmented SR 0.1', 'Augmented RI 0.1', 
                   'Augmented RS 0.1', 'Augmented RD 0.1', 'Augmented All 0.1', 'Augmented All 0.5']

acc_list = [Accuracy_orig, Accuracy_aug_sr, Accuracy_aug_ri, Accuracy_aug_rs, 
            Accuracy_aug_rd, Accuracy_aug_all_1, Accuracy_aug_all_5]

macro_f1_list = [Macro_F1_orig, Macro_F1_aug_sr, Macro_F1_aug_ri, Macro_F1_aug_rs, 
                 Macro_F1_aug_rd, Macro_F1_aug_all_1, Macro_F1_aug_all_5]

roc_auc_list = [ROC_AUC_orig, ROC_AUC_aug_sr, ROC_AUC_aug_ri, ROC_AUC_aug_rs, 
                ROC_AUC_aug_rd, ROC_AUC_aug_all_1, ROC_AUC_aug_all_5]

In [66]:
result_dict = {'Trial Name' : trial_name_list, 'Test Accuracy Score' : acc_list, 
               'Test Macro F1 Score' : macro_f1_list, 'Test ROC AUC Score' : roc_auc_list}

In [67]:
results_df = pd.DataFrame(result_dict)

results_df

,Trial Name,Test Accuracy Score,Test Macro F1 Score,Test ROC AUC Score
0,Original Data,0.703068,0.683048,0.766084
1,Augmented SR 0.1,0.703588,0.678805,0.764023
2,Augmented RI 0.1,0.700988,0.684669,0.768324
3,Augmented RS 0.1,0.710868,0.691864,0.773969
4,Augmented RD 0.1,0.693708,0.672480,0.760032
5,Augmented All 0.1,0.699948,0.681153,0.766542
6,Augmented All 0.5,0.698388,0.678707,0.763331


In [68]:
results_df.to_csv('./Saved_Models/EDA_b_10aug/All_DA_BERT_base_uncased_10aug.csv')

In [56]:
metrics_org_df = pd.DataFrame(metrics_orig)

metrics_org_df

,loss,accuracy,balanced_recall,balanced_precision,balanced_f1_score,auc_roc,val_loss,val_accuracy,val_balanced_recall,val_balanced_precision,val_balanced_f1_score,val_auc_roc
0,1.304218,0.413183,0.577113,0.382281,0.459615,0.587675,0.941717,0.536661,0.715030,0.468557,0.564904,0.727912
1,1.107177,0.478905,0.647966,0.427393,0.514761,0.662015,0.882533,0.579303,0.730828,0.510581,0.600173,0.765815
2,1.021858,0.517064,0.677782,0.456293,0.545109,0.699290,0.840768,0.616225,0.762120,0.526358,0.621697,0.789697
3,0.975327,0.548658,0.705791,0.475962,0.568248,0.727108,0.820926,0.622985,0.769488,0.540664,0.634129,0.800423
4,0.938600,0.565299,0.724991,0.487832,0.582859,0.741920,0.810832,0.635465,0.770686,0.540272,0.634159,0.808042
5,0.916459,0.579211,0.732874,0.494862,0.590459,0.753631,0.801379,0.644826,0.764099,0.550575,0.639085,0.813860
6,0.906114,0.584931,0.740733,0.499723,0.596443,0.758077,0.786131,0.641186,0.805583,0.558589,0.658708,0.812701
7,0.880125,0.601703,0.755497,0.507894,0.607100,0.769890,0.781943,0.654706,0.794092,0.549299,0.648268,0.822168
8,0.866558,0.607034,0.758863,0.515412,0.613523,0.776999,0.770042,0.652626,0.796694,0.559567,0.656503,0.823961
9,0.864026,0.609179,0.760102,0.514782,0.613530,0.777900,0.767062,0.664067,0.781519,0.570256,0.658559,0.830866


In [57]:
metrics_list = [metrics_orig, metrics_sr, metrics_ri, metrics_rs, metrics_rd, metrics_all_1, metrics_all_5]
name_list = ['fit_metrics_orig.csv', 'fit_metrics_sr.csv', 'fit_metrics_ri.csv', 'fit_metrics_rs.csv', 'fit_metrics_rd.csv', 'fit_metrics_all_1.csv', 'fit_metrics_all_5.csv']

i = 0
for m in metrics_list:
    df = pd.DataFrame(m)
    df.to_csv(name_list[i])
    i += 1

        